In [97]:
def add_h3_hexes(cab_data, aperture_size):
    """ Adds the h3 hex location designation to the cab data frame. """

    cab_data_w_h3_hex = (
        cab_data
        .assign(h3_hex=cab_data.apply(lambda x: h3.geo_to_h3(x.latitude, x.longitude, aperture_size), 1))
    )

    return cab_data_w_h3_hex

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons.
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v: v[0], polyline))
        lng.extend(map(lambda v: v[1], polyline))
        polylines.append(polyline)

    if folium_map is None:
        m = folium.Map(location=[sum(lat) / len(lat), sum(lng) / len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine = folium.PolyLine(locations=polyline, weight=8, color=color)
        m.add_child(my_PolyLine)
    return m


In [269]:
accidents_with_hexes = add_h3_hexes(accidents, 9)

In [270]:
m = visualize_hexagons(accidents_with_hexes.h3_hex.value_counts().head(20).index)
display(m)